In [ ]:
# ==================== Colab Setup ====================
# 在 Google Colab 中运行此 notebook 的设置

# 检测是否在 Colab 环境
try:
    import google.colab
    IN_COLAB = True
    print("✓ 运行在 Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ 运行在本地环境")

if IN_COLAB:
    import os
    import sys
    import subprocess
    from pathlib import Path

    project_clone_path = Path("/content/echemistpy")

    def run_and_check(cmd_list, start_msg, success_msg, failure_msg):
        """Helper function to run a subprocess command and check its return code."""
        print(f"\n{start_msg}...")
        result = subprocess.run(cmd_list, capture_output=True, text=True, check=False)
        if result.returncode != 0:
            print(f"✗ {failure_msg}，返回码: {result.returncode}")
            print("  stdout:", result.stdout)
            print("  stderr:", result.stderr)
            raise RuntimeError(f"{failure_msg}. Return code: {result.returncode}")
        print(f"✓ {success_msg}")
        return result

    # 1. 安装 uv 包管理器
    run_and_check(
        ["pip", "install", "-q", "uv"],
        "📦 安装 uv 包管理器",
        "uv 包管理器已安装",
        "安装 uv 包管理器失败"
    )

    # 2. 克隆项目 (cl_version 分支，包含所有数据)
    if not project_clone_path.exists():
        # 确保 Git LFS 已安装和初始化
        run_and_check(
            ["git", "lfs", "install"],
            "  ⚙️ 确保 Git LFS 已初始化",
            "Git LFS 已初始化",
            "Git LFS 初始化失败"
        )

        # 尝试禁用 GitHub HTTPS 凭据助手，以避免 'read Username' 错误（针对公共仓库）
        print("  ⚙️ 尝试禁用 GitHub HTTPS 凭据助手...")
        disable_credential_result = subprocess.run(
            ["git", "config", "--global", "credential.https://github.com.helper", ""],
            capture_output=True, text=True, check=False
        )
        if disable_credential_result.returncode != 0:
            print(f"✗ 禁用凭据助手失败，返回码: {disable_credential_result.returncode}")
            print("  stdout:", disable_credential_result.stdout)
            print("  stderr:", disable_credential_result.stderr)
            # 不抛出错误，即使失败也可能继续尝试克隆
        else:
            print("  ✓ GitHub HTTPS 凭据助手已禁用")

        git_command_list = ["git", "clone", "--branch", "cl_version", "https://github.com/liuchzzyy/echemistpy.git", str(project_clone_path)]
        run_and_check(
            git_command_list,
            "📥 克隆项目 (cl_version 分支，包含所有数据)",
            f"项目已克隆到 {project_clone_path}",
            "Git 克隆命令执行失败"
        )

        if not project_clone_path.exists():
            raise FileNotFoundError(f"✗ 项目克隆失败，目录未创建: {project_clone_path}")
    else:
        print("\n✓ 项目已存在，跳过克隆。")

    # 3. 切换到项目目录并添加到 Python 路径
    if project_clone_path.exists():
        os.chdir(project_clone_path)
        sys.path.insert(0, str(project_clone_path / "src"))
        print(f"\n✓ 环境配置完成！当前工作目录: {os.getcwd()}")
        print(f"✓ 'echemistpy/src' 已添加到 Python 路径。")

        # 4. 使用 uv 安装依赖
        run_and_check(
            ["uv", "pip", "install", "-q", "numpy==1.26.0", "scipy==1.11.0"],
            "📦 使用 uv 安装依赖 (修复 numpy/scipy 兼容性)",
            "兼容的 numpy/scipy 已安装",
            "uv 安装兼容的 numpy/scipy 失败"
        )
        run_and_check(
            ["uv", "pip", "install", "-q", "-e", "."],
            "📦 使用 uv 安装剩余依赖",
            "剩余依赖已安装",
            "uv 安装剩余依赖失败"
        )

        print("\n💡 提示: 下一步将从 OneDrive 自动下载测试数据（约 830 MB）")
    else:
        print("\n✗ 无法完成环境配置，项目目录不存在。")


✓ 运行在 Google Colab

📦 安装 uv 包管理器...
✓ uv 包管理器已安装

✓ 项目已存在，跳过克隆。

✓ 环境配置完成！当前工作目录: /content/echemistpy
✓ 'echemistpy/src' 已添加到 Python 路径。

📦 使用 uv 安装依赖 (修复 numpy/scipy 兼容性)...


# 🚀 在 Google Colab 中运行此 Notebook

## 快速开始

1. **在 Colab 中打开此 notebook**：
   - 访问 [Google Colab](https://colab.research.google.com/)
   - `文件` → `打开 Notebook` → `GitHub`
   - 输入：`liuchzzyy/echemistpy`
   - 选择分支：`cl_version`
   - 选择：`notebooks/test_stxm_analyzer.ipynb`

2. **依次运行单元格**：
   - 第 2 个单元格：安装环境（约 1-2 分钟）
   - 第 5 个单元格：下载数据（约 3-5 分钟，830 MB）
   - 后续单元格：执行分析

3. **数据来源**：
   - 自动从 OneDrive/SharePoint 下载
   - 无需手动上传大文件

## 💡 注意事项

- **数据下载**: 测试数据约 830 MB，下载需要几分钟
- **运行时间**: 完整分析约需 5-10 分钟
- **GPU**: 不需要（但可以加速某些操作）

---

# Test STXMAnalyzer with Real Data

This notebook tests the `STXMAnalyzer` in `src/echemistpy/processing/analyzers/stxm.py` using a real TXM HDF5 file.

In [13]:
import sys
import os
from pathlib import Path
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# 检测是否在 Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 设置项目路径
if IN_COLAB:
    # Colab 中，假设已经 cd 到项目根目录
    project_root = Path.cwd()
else:
    # 本地环境，notebook 在 notebooks/ 子目录
    project_root = Path("..").resolve()
    if str(project_root / "src") not in sys.path:
        sys.path.append(str(project_root / "src"))

# Reload modules to pick up changes
import importlib
if 'echemistpy.processing.analyzers.stxm' in sys.modules:
    importlib.reload(sys.modules['echemistpy.processing.analyzers.stxm'])

from echemistpy.io.loaders import load
from echemistpy.processing.analyzers.stxm import STXMAnalyzer

print(f"✓ 项目根目录: {project_root}")

ImportError: cannot import name '_center' from 'numpy._core.umath' (/usr/local/lib/python3.12/dist-packages/numpy/_core/umath.py)

## 下载数据文件

在 Colab 中从 OneDrive/SharePoint 下载测试数据。

In [ ]:
# ==================== 下载数据文件 ====================
import urllib.request
import shutil
from pathlib import Path

# OneDrive/SharePoint 数据链接（转换为直接下载链接）
ONEDRIVE_URL = "https://uab-my.sharepoint.com/:u:/g/personal/1615992_uab_cat/IQCiwUxTb7I-QpG_3-5KDu3VAZQiEam_jrJOLEVC0rDR6vk?e=rsxesw&download=1"

# 目标路径
data_dir = project_root / "docs" / "examples" / "TXM" / "αMnO2"
data_file = data_dir / "20230629_E1A_749.7x177.5y_specnorm_aliOF.hdf5.hdf5"

# 检查文件是否已存在
if data_file.exists():
    print(f"✓ 数据文件已存在: {data_file}")
    print(f"  大小: {data_file.stat().st_size / 1024 / 1024:.2f} MB")
else:
    print(f"📥 从 OneDrive 下载数据...")
    print(f"  URL: {ONEDRIVE_URL[:80]}...")

    # 创建目录
    data_dir.mkdir(parents=True, exist_ok=True)

    try:
        # 下载文件
        print("  下载中... (这可能需要几分钟，文件约 830 MB)")

        # 使用临时文件避免部分下载
        temp_file = data_file.with_suffix('.tmp')

        def download_progress(block_num, block_size, total_size):
            downloaded = block_num * block_size
            if total_size > 0:
                percent = min(downloaded * 100 / total_size, 100)
                mb_downloaded = downloaded / 1024 / 1024
                mb_total = total_size / 1024 / 1024
                print(f"\r  进度: {percent:.1f}% ({mb_downloaded:.1f}/{mb_total:.1f} MB)", end='')

        urllib.request.urlretrieve(ONEDRIVE_URL, temp_file, download_progress)
        print()  # 换行

        # 移动到最终位置
        shutil.move(str(temp_file), str(data_file))

        print(f"✓ 下载完成: {data_file}")
        print(f"  大小: {data_file.stat().st_size / 1024 / 1024:.2f} MB")

    except Exception as e:
        print(f"✗ 下载失败: {e}")
        print("\n备用方案:")
        print("1. 手动下载文件:")
        print(f"   {ONEDRIVE_URL.replace('&download=1', '')}")
        print("2. 使用 Google Drive 上传数据")
        print("3. 上传到 Colab:")
        print("   from google.colab import files")
        print("   uploaded = files.upload()")

        # 清理临时文件
        if temp_file.exists():
            temp_file.unlink()

        raise

In [ ]:
# ==================== 加载数据 ====================
# 使用 alpha MnO2 示例数据
print(f"📂 加载数据: {data_file}")

if not data_file.exists():
    raise FileNotFoundError(f"数据文件不存在: {data_file}\n请先运行上面的下载单元格！")

raw_data, raw_info = load(data_file)

print("\n✓ 数据加载成功")
print(f"\n{'='*50}")
print("Raw Data Info:")
print(f"{'='*50}")
print(raw_info.to_dict())

print(f"\n{'='*50}")
print("Raw Data Structure:")
print(f"{'='*50}")
print(raw_data.data)

In [ ]:
# 2. Analyze
analyzer = STXMAnalyzer(
    pca_components=6,
    perform_clustering=True,
    n_clusters=4,
    # clustering_method="kmeans",
    perform_fitting=True,
)

# Use analyze() method which handles validation, preprocessing, computation, and metadata inheritance
result_data, result_info = analyzer.analyze(raw_data, raw_info)

# 3. Inspect Results
print("\nAnalysis Info Parameters:")
print(result_info.parameters)

print("\nResult Dataset:")
ds_res = result_data.data
print(ds_res)


In [ ]:
# 4. Plot
ds_res = result_data.data

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# PCA Denoised Map (Sum)
if "denoised" in ds_res:
    if "energy_eV" in ds_res.denoised.dims:
        img_sum = ds_res.denoised.sum(dim="energy_eV")
        ax[0].imshow(img_sum, cmap="viridis")
        ax[0].set_title("Denoised Sum")
    else:
        ax[0].text(0.5, 0.5, "Denoised has no energy_eV dim")
elif "optical_density" in ds_res:
    if "energy_eV" in ds_res.optical_density.dims:
        img_sum = ds_res.optical_density.sum(dim="energy_eV")
        ax[0].imshow(img_sum, cmap="viridis")
        ax[0].set_title("Original Sum")
else:
    ax[0].text(0.5, 0.5, "No Image Data")

# Cluster Labels
if "cluster_labels" in ds_res:
    ax[1].imshow(ds_res.cluster_labels, cmap="tab10")
    ax[1].set_title("Cluster Labels")
else:
    ax[1].text(0.5, 0.5, "No Cluster Labels")

# Cluster Centroids / Spectra
if hasattr(result_info, "others") and "cluster_centroids" in result_info.others:
    centroids = result_info.others["cluster_centroids"]
    if "energy_eV" in ds_res.coords:
        energy = ds_res.energy_eV
        for i, cent in enumerate(centroids):
            ax[2].plot(energy, cent, label=f"Cluster {i}")

        ax[2].legend()
        ax[2].set_title("Cluster Spectra")
    else:
        ax[2].text(0.5, 0.5, "No energy_eV axis")
else:
    ax[2].text(0.5, 0.5, "No centroids")

plt.tight_layout()
plt.show()


# Task
Update cell `2d5142c6` to use `subprocess.run` for the `git clone` command, capturing and printing `stdout` and `stderr` to diagnose the `32768` return code.

## Update Setup Cell with Subprocess

### Subtask:
Modify cell `2d5142c6` to use `subprocess.run` for the `git clone` command to capture more detailed error output, aiming to diagnose the `32768` return code.


## Summary:

### Data Analysis Key Findings
No data analysis has been performed yet. The current task focuses on preparing a diagnostic step.

### Insights or Next Steps
*   The next step is to modify cell `2d5142c6` to use `subprocess.run` for the `git clone` command. This modification is intended to capture the standard output (`stdout`) and standard error (`stderr`) to diagnose the `32768` return code.


# Task
Modify cell `2d5142c6` to set the environment variable `GIT_TERMINAL_PROMPT=0` for the `git clone` subprocess call, which should resolve the 'could not read Username' error during repository cloning.

## Modify git clone command with GIT_TERMINAL_PROMPT=0

### Subtask:
Modify cell `2d5142c6` to set `GIT_TERMINAL_PROMPT=0` in the environment for the `git clone` subprocess call, addressing the persistent 'could not read Username' error.


## Summary:

### Data Analysis Key Findings
*   The `git clone` operation encountered a persistent "could not read Username" error, indicating a prompt for user credentials in a non-interactive environment.
*   The issue was addressed by setting the `GIT_TERMINAL_PROMPT=0` environment variable for the `git clone` subprocess call, which suppresses interactive prompts.

### Insights or Next Steps
*   When executing Git commands in automated scripts or non-interactive environments, it is crucial to set `GIT_TERMINAL_PROMPT=0` to prevent hangs or failures caused by credential prompts.
*   The immediate next step is to execute the modified cell to confirm that the `git clone` command now proceeds successfully without the "could not read Username" error.
